Строим запрос к нашему сервису

In [71]:
import requests

import urllib.request
import json 

In [72]:
# Пример данных
data = ( 
    "Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them.\xa0As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity. \xa0Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.",
    "ustwo offers you the opportunity to be yourself, whilst delivering the best work on the planet for some of the biggest and most innovative brands. A culture thriving on collaboration underpins what is an amazing work smart/ live well environment.We genuinely care about the work that we deliver and the people who help make it all possible. We only invest in projects, people and practices that we believe in, to ensure we remain excited about every opportunity.",
    "We are negotiable on salary and there is the potential for equity for the right candidate.",
    "We don’t care where you studied or what your GPA in college was. But we do want someone who is:Strong with Python and No SQL DatabasesEntrepreneurial--independent and can solve problems without being micro-managedCan brainstorm multiple approaches to solving problems instead of being attached to just one way of doing thingsIcing on the cakePrevious experience with Google Appengine’s Datastore (NDB)Previous experience at e-commerce and app-based startupsFull Stack Development Experience from the Frontend (either with apps or html) all the way to the linux server.An understanding of algorithms and experience with libraries to implement themExperience with machine learning and natural language processing"
)

In [73]:
# формируем запрос
def send_json(x):
    description, company_profile, benefits, requirements = x
    #print(description, company_profile, benefits)
    body = {
        'description': description, 
        'company_profile': company_profile,
        'benefits': benefits,
        'requirements': requirements
        }
    #myurl = 'http://948a-35-230-58-89.ngrok.io/' + '/predict'
    myurl = 'http://127.0.0.1:8082/' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [74]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

предсказание 0.01


Сделаем обработку массы запросов:

1. загрузим данные X_test

In [75]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

X_test.fillna('', inplace=True)

In [76]:
X_test[['description', 'company_profile', 'benefits', 'requirements']]

,description,company_profile,benefits,requirements
0,Stylect is a dynamic startup that helps helps ...,,We are negotiable on salary and there is the p...,We don’t care where you studied or what your G...
1,General Summary: Achieves maximum sales profit...,,Great Health and DentalFast Advancement Opport...,
2,"At ustwo™ you get to be yourself, whilst deliv...",ustwo offers you the opportunity to be yoursel...,,Skills• Experience interfacing directly with c...
3,About EDITDEDITD runs the world's biggest appa...,"We build software for fashion retailers, to he...",,Required Skills / Experience:Ability to analys...
4,As a Web Engineer at Runscope you'll be respon...,Runscope is building tools for developers work...,Be a part of an experienced team who have work...,Extensive front-end web experience (HTML/CSS/J...
...,...,...,...,...
5896,Fabrication and Printing Company in Long Islan...,,Excellent opportunity to learn exci...,- You must be proficient in Photoshop a...
5897,Serve as the primary lead and project manager ...,"Palerra, Inc. designed LORIC™ to protect an en...",What's In It For You?Competitive compensation ...,"5-6 years running, managing and operating mark..."
5898,"Jiffy, a world wide leader in mobile applicati...",Jiffy Worldwide is the parent company of the J...,,iOS Developer should be very familiar and expe...
5899,What our client needs…A Medical Director who i...,Human capital is usually the biggest asset and...,,What you need to have….Doctor of Medicine (MD)...


2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [77]:
response = send_json(X_test[['description', 'company_profile', 'benefits', 'requirements']].iloc[0,:])

In [78]:
response

0.04

3. Сделаем N запросов и оценим время

In [86]:
N = 500

In [87]:
%%time
predictions = X_test[['description', 'company_profile', 'benefits', 'requirements']].iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: user 757 ms, sys: 33.8 ms, total: 791 ms
Wall time: 7.54 s


In [88]:
predictions.values[:5]

array([0.04, 0.  , 0.  , 0.  , 0.  ])

4. Посчитаем метрику

In [89]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.19, F-Score=0.923, Precision=1.000, Recall=0.857


In [90]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.991698976041356